### Eventloop
Ein Eventloop is eine Schleife, die z.B. all 100 Millisekunden prüft, ob
neue Events ( Tastendrücke, Mausklicks, Mausbewegungen, ...) stattgefunden haben und dann
entsprechend reagiert.

Wir benutzen einen Eventloop, um ein einfaches Spiel zu programmieren,
bei dem der Spieler einen Linienzug zeichnet. Die Linie setzt sich jeweils ein paar Pixel in
Richtung der zuletzt gedrücken Pfeiltaste fort.
Kreuzt sich die Linie oder verlässt sie einen bestimmten Bereich, endet das Spiel.

Der Spielzustand ist als NxN Tabelle repräsentiert.
Bereits besuchte Felder sind mit `True` markier, leere mit `False`.
Die Spielkomponente hat eine Funktion `move(dx, dy)`, welche das aktuelle Feld `(x, y)`,
auf `(x+dx, y+dy)` updated.

Der Spielzustand wird auf der Leinwand dargestellt. Dabei wird in der mitte der Feldes eine
Linie in Bewegungsrichtung gezeichnet.

Wird eine Pfeiltaste gedrückt, wird diese als erstes Element in eine Liste
`event_queue` eingefügt. 
Ist der Eventqueue  nicht leer, entfernt 
der Eventloop die zuletzt gedrückt Taste, andernfalls wird
die zuletzt gedrückt Taste beibehalten. Die Taste wird in der Variable `key` gespeichert.
Dann wird die Funktione `game.move` mit den Argumenten `key_move[key]` aufgerufen.

Der Event-Loop wird mit Hilfe der Funktion `threading.Timer`.
Nachstehend definierte Funktion `event_loop` 
macht sich zu einem Thread, der mit etwas Verzögerung startet.
Und dieser Thread macht aus der Funktion erneut einen solchen Thread.
Dieser Vorgang wird gestoppt, wenn das `stop_event` gesetzt wird.


```python
event_queue = ['ArrowUp', 'ArrowRight']

key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }
            
def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()            
```


***
**threading.Event**:  
Ein `threading.Event` ist im wesentlichen ein Flag, das entweder `True` oder `False` ist.
Es hat folgende Methoden:
- `stop_event = threading.Event()`: Erstellt ein `threading.Event`,
- `is_set()` : liefert `True` oder `False` (Default),
- `set()`    : `is_set()` liefert nun `True`,
- `clear()`  : `is_set()` liefert nun `False`.
***

In [3]:
import threading


stop_event = threading.Event()
stop_event

<threading.Event at 0x7f463a345490: unset>

In [ ]:
stop_event.is_set()

In [ ]:
stop_event.set()
stop_event.is_set()

In [ ]:
stop_event.clear()
stop_event.is_set()

***
Die Spielkomponente unseres Spiels ist in `game.py` implementiert
***

In [1]:
import game

# Right, Up, Left, Down -> Game over
moves = [(1, 0), (0, 1), (-1, 0)] + [(0, -1)]

game.new_game(nrow=5, ncol=5)
for dx, dy in moves:
    game.move(dx, dy)

game.state

event: new_game, kwargs: {'head': [2, 2], 'size': (5, 5)}
event: move, kwargs: {'head': [3, 2], 'size': (5, 5)}
event: move, kwargs: {'head': [3, 3], 'size': (5, 5)}
event: move, kwargs: {'head': [2, 3], 'size': (5, 5)}
event: move, kwargs: {'head': [2, 2], 'size': (5, 5)}
event: game_over, kwargs: {}


{'size': (5, 5),
 'body': [[False, False, False, False, False],
  [False, False, False, False, False],
  [False, False, True, True, False],
  [False, False, True, True, False],
  [False, False, False, False, False]],
 'head': [2, 2],
 'game_over': True}

***
Eventloop testen.
***

In [2]:
import threading


key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }

stop_event = threading.Event()
event_queue = []


def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()

In [3]:
game.new_game()

event: new_game, kwargs: {'head': [9, 9], 'size': (19, 19)}


In [4]:
event_queue = ['ArrowUp'] + ['ArrowRight'] * 3
stop_event.clear()
event_loop()

event: move, kwargs: {'head': [10, 9], 'size': (19, 19)}
event: move, kwargs: {'head': [11, 9], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 9], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 8], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 7], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 6], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 5], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 4], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 3], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 2], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 1], 'size': (19, 19)}
event: move, kwargs: {'head': [12, 0], 'size': (19, 19)}
event: game_over, kwargs: {}


In [7]:
# laeuft der Eventloop Thred noch?
[thread.name for thread in threading.enumerate()[8:]]

[]

In [6]:
# Eventloop stoppen
stop_event.set()

In [2]:
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        dx, dy = key_move[key]
        game.move(dx, dy)
        move(game.state['head'], game.state['size'])
        if game.is_game_over():
            game_over()
    if key == 'n':
        game.new_game()
        new_game(game.state['head'], game.state['size'])


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


canvas.on_key_down(on_key_down)
display(canvas, out)
canvas.focus()

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

(1, 0)

In [3]:
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        dx, dy = key_move[key]
        game.move(dx, dy)
    if key == 'n':
        game.new_game()


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


def update(event, **kwargs):
    if event == 'new_game':
        new_game(**kwargs)
    if event == 'game_over':
        game_over()
    if event == 'move':
        move(**kwargs)


game.update = update
canvas.on_key_down(on_key_down)
display(canvas, out)
canvas.focus()

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [10]:
threading.active_count()

8

In [5]:
import threading
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }

stop_event = threading.Event()
event_queue = []


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        event_queue.insert(0, key)
    if key == 'n':
        game.new_game()


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


def update(event, **kwargs):
    out.append_stdout(f'{event}, {kwargs}')
    if event == 'new_game':
        new_game(**kwargs)
        stop_event.clear()
        event_loop()
    if event == 'game_over':
        game_over()
        stop_event.set()
    if event == 'move':
        move(**kwargs)


def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()


game.update = update
canvas.on_key_down(on_key_down)

display(canvas, out)
canvas.focus()

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [6]:
threading.active_count()

8

In [ ]:
from ipywidgets import Button


button_stop = Button(description='Stop')
button_stop.on_click(lambda bt: stop_event.set())

### Aufgabe
Erstellen einen weitere Button `New Game`. Beim Klicken auf diesen Button soll eine 
Funktion `new_game()` aufgerufen werden, welche
- die Leinwand löscht,
- `stop_event.clear()` aufruft,
- den Event-Queue gelöscht,
- `'Right'` in Event-Queue einfügt,
- den Event-Loop startet und der Leinwand Fokus gibt.  

Leite Fehlermeldungen der Funktion `new_game()` ins Output-Widget `out` um.